# Knowledge Booster

## Description:

Knowledge Booster is an AI-powered educational assistant that uses CrewAI agents enhanced with Gemini, Cohere, and Serper tools to find high-quality resources—books, podcasts, and YouTube videos—on any topic you choose. Whether you're diving into artificial intelligence or brushing up on football history, this app curates tailored content from across the web and presents it in a neatly formatted markdown report.

In [ ]:
# !pip install -U crewai crewai_tools langchain_community cohere google-generativeai

In [2]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')
sys.setrecursionlimit(5000)


### Just for test APIS & connections

In [3]:
#API KEYS
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get("gemini_api")
os.environ["COHERE_API_KEY"] = userdata.get("cohere_api")
os.environ["SERPER_API_KEY"] = userdata.get("serper_api")

#CONFIGURE APIS
import google.generativeai as genai
import cohere

genai.configure(api_key=os.environ["GEMINI_API_KEY"])
co = cohere.Client(os.environ["COHERE_API_KEY"])

from crewai import Crew, Agent, Task, LLM
from crewai_tools import SerperDevTool
from crewai.tools import BaseTool

# Test APIs
print("Gemini:", genai.GenerativeModel('gemini-1.5-flash').generate_content("Hello!").text)
print("Cohere:", co.generate(model='command', prompt="Hi from cohere!", max_tokens=20).generations[0].text)


Gemini: Hello there! How can I help you today?

Cohere:  Hi there! I'm your friendly AI-assistant chatbot, and I'm ready to answer your questions


### LLMs and Tools

In [42]:
# LLMs
gemini_llm = LLM(provider="google_ai_studio", model="gemini/gemini-1.5-flash", api_key=os.environ["GEMINI_API_KEY"])
cohere_llm = LLM(provider="cohere", model="command", api_key=os.environ["COHERE_API_KEY"])

# Tools
search_tool = SerperDevTool()

### Agents

In [43]:
# Agents
book_agent = Agent(
    role="Book Finder",
    goal="Find informative and free books related to a given topic.",
    backstory="You help users expand their knowledge by recommending relevant books (PDFs or from platforms like Zlibrary or PDF Drive).",
    llm=gemini_llm,
    tools=[search_tool],
    allow_delegation=False
)

podcast_agent = Agent(
    role="Podcast Curator",
    goal="Find engaging podcasts that match the user's learning goals.",
    backstory="You're great at spotting valuable English podcasts in the user's topic.",
    llm=gemini_llm,
    tools=[search_tool],
    allow_delegation=False
)

youtube_agent = Agent(
    role="YouTube Explorer",
    goal="Find educational YouTube videos and summaries for a topic.",
    backstory="You help users learn visually by recommending the best educational videos available.",
    llm=gemini_llm,
    tools=[search_tool],
    allow_delegation=False
)


### Tasks

In [44]:
book_task = Task(
    description="Find 2 books or PDFs for topic: {topic}. Include title and access link if possible.",
    expected_output="List of books with links.",
    agent=book_agent
)

podcast_task = Task(
    description="Find 2 podcast episodes related to topic: {topic}. Include link + short description.",
    expected_output="Podcast links + short context.",
    agent=podcast_agent
)

youtube_task = Task(
    description="Find 2 YouTube videos that explain {topic} in English. Prefer summaries or simple explainers.",
    expected_output="List of videos with links and why they were selected.",
    agent=youtube_agent
)


### Crew AI

In [45]:
# Crew
crew = Crew(
    agents=[book_agent, podcast_agent, youtube_agent],
    tasks=[book_task, podcast_task, youtube_task],
    verbose=False,
    memory=False
)




In [46]:
# Run Crew
inputs = {
    "topic": "AI"
}


### output

In [47]:
result = crew.kickoff(inputs=inputs)
print("FINAL SUMMARY RESULT")
print("\n")
print(result)

FINAL SUMMARY RESULT


1. **Google's AI Course for Beginners (in 10 minutes)!** - https://www.youtube.com/watch?v=Yq0QkCxoTHM&pp=0gcJCdgAo7VqN5tD

   This video is selected because it's explicitly designed for beginners and promises a concise (10-minute) overview of AI.  The title and description suggest a simple explanation of fundamental concepts.


2. **99% of Beginners Don't Know the Basics of AI** - https://www.youtube.com/watch?v=nVyD6THcvDQ&pp=0gcJCdgAo7VqN5tD

   This video is chosen for its targeted audience ("beginners") and the intriguing title, suggesting it will address common misconceptions and provide a foundational understanding of AI.


In [48]:
print(result.tasks_output)

[TaskOutput(description='Find 2 books or PDFs for topic: AI. Include title and access link if possible.', name=None, expected_output='List of books with links.', summary='Find 2 books or PDFs for topic: AI. Include title...', raw='1. **Artificial Intelligence: A Modern Approach:** [https://people.engr.tamu.edu/guni/csce625/slides/AI.pdf](https://people.engr.tamu.edu/guni/csce625/slides/AI.pdf)\n\n2. **Artificial Intelligence for Everyone:** [https://www.researchgate.net/publication/338843576_Artificial_Intelligence_for_Everyone](https://www.researchgate.net/publication/338843576_Artificial_Intelligence_for_Everyone)', pydantic=None, json_dict=None, agent='Book Finder', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='Find 2 podcast episodes related to topic: AI. Include link + short description.', name=None, expected_output='Podcast links + short context.', summary='Find 2 podcast episodes related to topic: AI. Include link...', raw='The provided search results offer m

In [51]:
def parse_crew_results(result):
    outputs = {}
    for task_output in result.tasks_output:
        # Use the agent role or description as key
        key = task_output.agent.lower().replace(" ", "_")
        outputs[key] = task_output.raw
    return outputs


In [50]:
outputs = parse_crew_results(result)

print("Books:\n", outputs[0])
print("Podcasts:\n", outputs[1])
print("YouTube Videos:\n", outputs[2])

Books:
 1. **Artificial Intelligence: A Modern Approach:** [https://people.engr.tamu.edu/guni/csce625/slides/AI.pdf](https://people.engr.tamu.edu/guni/csce625/slides/AI.pdf)

2. **Artificial Intelligence for Everyone:** [https://www.researchgate.net/publication/338843576_Artificial_Intelligence_for_Everyone](https://www.researchgate.net/publication/338843576_Artificial_Intelligence_for_Everyone)
Podcasts:
 The provided search results offer many podcast options, but lack specific episode links.  To provide a complete answer with links and descriptions, more information is needed, such as specific podcasts to check for relevant episodes.  I need more direction.  For example, specifying a podcast from the list above would enable me to provide specific episodes and descriptions.
YouTube Videos:
 1. **Google's AI Course for Beginners (in 10 minutes)!** - https://www.youtube.com/watch?v=Yq0QkCxoTHM&pp=0gcJCdgAo7VqN5tD

   This video is selected because it's explicitly designed for beginners 

#### another trial

In [39]:
# result = crew.kickoff(inputs={"topic": "football"})
# outputs = parse_crew_results(result)

In [38]:
print("Books:\n", outputs[0])
print("Podcasts:\n", outputs[1])
print("YouTube Videos:\n", outputs[2])

Books:
 1. **Title:** Football - Profile Books
**Link:** https://profilebooks.com/wp-content/uploads/wpallimport/files/PDFs/9781781259221_preview.pdf

2. **Title:** Football Stories - World Book Day
**Link:** https://www.worldbookday.com/wp-content/uploads/2021/02/Football-Stories_compressed.pdf
Podcasts:
 **Podcast 1:**

* **Title:**  Micah's Star-Studded Trip, Acting Tips From Tom Cruise & Big Early Moves In The Transfer Window
* **Link:** https://open.spotify.com/show/2fDn3EgvJZ5J1k5rrBwrlZ (This link goes to the main podcast page; individual episode links are unavailable without direct access to Spotify).
* **Short Description:** This episode of "The Rest Is Football" likely features Gary Lineker, Alan Shearer, and Micah Richards discussing Micah's experiences, Tom Cruise's acting advice, and major transfer news in the football world.

**Podcast 2:**

* **Title:** Justin Kluivert: Embracing The Expectations, The ... (Title is incomplete from search results)
* **Link:** https://open

### Deployment with Gradio

In [68]:
import gradio as gr
from datetime import datetime
import re
import os

def convert_to_markdown_links(text):

    lines = text.strip().splitlines()
    md_links = []
    for line in lines:
        url_match = re.search(r'(https?://\S+)', line)
        if url_match:
            url = url_match.group(1)
            title = line.replace(url, "").strip(" -•–:")
            md_links.append(f"- [{title if title else url}]({url})")
        else:
            if line.strip():
                md_links.append(f"- {line.strip()}")
    return "\n".join(md_links) if md_links else "No results found."

# --- CrewAI output parsing ---
def parse_crew_results(result):
    outputs = {
        "books_results": "No books found.",
        "podcasts_results": "No podcasts found.",
        "youtube_results": "No YouTube videos found.",
    }

    for task_output in result.tasks_output:
        agent_name = task_output.agent.lower()
        if "book" in agent_name:
            outputs["books_results"] = task_output.raw.strip()
        elif "podcast" in agent_name:
            outputs["podcasts_results"] = task_output.raw.strip()
        elif "youtube" in agent_name:
            outputs["youtube_results"] = task_output.raw.strip()

    return outputs["books_results"], outputs["podcasts_results"], outputs["youtube_results"]

# --- Gradio handler ---
def run_crew_interface(topic):
    if not topic:
        return "Please enter a topic.", "", "", None

    inputs = {"topic": topic}
    result = crew.kickoff(inputs=inputs)

    if not hasattr(result, "tasks_output") or not result.tasks_output:
        return "No result returned from Crew.", "", "", None

    books_raw, podcasts_raw, youtube_raw = parse_crew_results(result)

    # Convert to clickable Markdown links
    books_md = convert_to_markdown_links(books_raw)
    podcasts_md = convert_to_markdown_links(podcasts_raw)
    youtube_md = convert_to_markdown_links(youtube_raw)

    # Markdown file content
    markdown_report = f"""# 📘 Knowledge Booster Report

**Topic:** {topic}
**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

---

## 📚 Books
{books_md}

---

## 🎧 Podcasts
{podcasts_md}

---

## 📺 YouTube Videos
{youtube_md}
"""

    # Save file
    file_path = f"{topic.replace(' ', '_')}_report.md"
    with open(file_path, "w") as f:
        f.write(markdown_report)

    return books_md, podcasts_md, youtube_md, file_path

# --- Build Gradio Interface ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🌟 Knowledge Booster with CrewAI\nGet books, podcasts, and videos on any topic.")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a Topic", placeholder="e.g., Artificial Intelligence", lines=1)
        # run_btn = gr.Button("🔍 Search")

    with gr.Row():
       run_btn = gr.Button("🔍 Search")


    with gr.Row():

        gr.Markdown("## 📚 Books")
        gr.Markdown("## 🎧 Podcasts")
        gr.Markdown("## 📺 YouTube Videos")


    with gr.Row():
        books_out = gr.Markdown(label="📚 Books")
        podcasts_out = gr.Markdown(label="🎧 Podcasts")
        youtube_out = gr.Markdown(label="📺 YouTube Videos")


    run_btn.click(
          fn=run_crew_interface,
          inputs=topic_input,
          outputs=[books_out, podcasts_out, youtube_out]
      )

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c6456613afce190cd0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
